In [1]:
import dagshub
dagshub.init(repo_owner='sahilbhardwaj23', repo_name='Youtube-comment-Sentiment-Analysis', mlflow=True)

Accessing as sahilbhardwaj23

Initialized MLflow to track repo "sahilbhardwaj23/Youtube-comment-Sentiment-Analysis"

Repository sahilbhardwaj23/Youtube-comment-Sentiment-Analysis initialized!

In [2]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("https://dagshub.com/sahilbhardwaj23/Youtube-comment-Sentiment-Analysis.mlflow")

In [3]:
from mlflow.tracking import MlflowClient
import mlflow

In [4]:
def set_or_create_experiment(experiment_name):
    client = MlflowClient()
    experiment = client.get_experiment_by_name(experiment_name)

    # Check if the experiment exists and is active
    if experiment:
        if experiment.lifecycle_stage == "active":
            print(f"Using existing experiment '{experiment_name}' (ID: {experiment.experiment_id})")
            mlflow.set_experiment(experiment_name)
        else:
            # Restore if it's deleted
            print(f"Restoring deleted experiment '{experiment_name}' (ID: {experiment.experiment_id})")
            client.restore_experiment(experiment.experiment_id)
            mlflow.set_experiment(experiment_name)
    else:
        # Create a new experiment if it doesn't exist
        print(f"Creating new experiment '{experiment_name}'")
        mlflow.set_experiment(experiment_name)



In [5]:
# Set or create an experiment
mlflow.set_experiment("ML Algos with HP Tuning")

<Experiment: artifact_location='mlflow-artifacts:/eea9feb5bf974d4caa1db1d0e2e46f19', creation_time=1730567889441, experiment_id='7', last_update_time=1730567889441, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

e:\Projects\Youtube Comment Analysis\Youtube commnent Plugin\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
df = pd.read_csv('reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [8]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for SVM

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Modify log_mlflow to accept additional params argument
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params=None):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Log additional parameters (e.g., hyperparameters)
        if params:
            for param, value in params.items():
                mlflow.log_param(param, value)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")



# Step 6: Optuna objective function for SVM
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for SVM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=15)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)

    # Log the best model with MLflow, including best_params
    log_mlflow("SVM", best_model, X_train, X_test, y_train, y_test, params=best_params)

# Run the experiment for SVM
run_optuna_experiment()


[I 2024-11-03 08:18:55,395] A new study created in memory with name: no-name-a6b3e36d-6cc1-4d11-a54e-5d7f077bfb43
[I 2024-11-03 08:25:52,758] Trial 0 finished with value: 0.4769604734728387 and parameters: {'C': 0.02175382023873877, 'kernel': 'rbf'}. Best is trial 0 with value: 0.4769604734728387.
[I 2024-11-03 08:33:00,904] Trial 1 finished with value: 0.3839568801521877 and parameters: {'C': 0.010114896195834713, 'kernel': 'poly'}. Best is trial 0 with value: 0.4769604734728387.
[I 2024-11-03 08:54:23,527] Trial 2 finished with value: 0.7109490593954766 and parameters: {'C': 0.8803522358234165, 'kernel': 'poly'}. Best is trial 2 with value: 0.7109490593954766.
[I 2024-11-03 09:01:55,535] Trial 3 finished with value: 0.3628196998520397 and parameters: {'C': 0.004613146632322706, 'kernel': 'poly'}. Best is trial 2 with value: 0.7109490593954766.
[I 2024-11-03 09:08:19,979] Trial 4 finished with value: 0.5656309448319594 and parameters: {'C': 0.08860031386362646, 'kernel': 'poly'}. Best